In [13]:
import cv2  # Import OpenCV
import numpy as np

def match_histograms(source, reference):
    """
    Adjust the pixel values of a grayscale image such that its histogram
    matches that of a target image
    """
    source_hist, bins = np.histogram(source.flatten(), 256, [0, 256])
    reference_hist, _ = np.histogram(reference.flatten(), 256, [0, 256])

    # Cumulative distribution function
    source_cdf = np.cumsum(source_hist) 
    reference_cdf = np.cumsum(reference_hist)

    # Create a lookup table to map pixel values
    lookup_table = np.zeros(256, dtype=np.uint8)
    for i in range(256):
        idx = np.abs(source_cdf[i] - reference_cdf).argmin()
        lookup_table[i] = idx

    # Apply the mapping to get the result image
    result = cv2.LUT(source, lookup_table)
    return result

def adjust_color_space(video_path, image_path):
    # Read the first frame from the video
    video = cv2.VideoCapture(video_path)
    success, frame_from_video = video.read()
    video.release()

    if not success:
        print("Error: Could not read the first frame")
        return

    # Convert the frame to RGB
    frame_from_video_rgb = cv2.cvtColor(frame_from_video, cv2.COLOR_BGR2RGB)

    # Read and convert the image to RGB
    image_rgb = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    # Match histograms for each channel
    adjusted_image = np.zeros_like(image_rgb)
    for channel in range(3):
        adjusted_image[..., channel] = match_histograms(image_rgb[..., channel], frame_from_video_rgb[..., channel])

    # Scale the image to match video frame's resolution
    adjusted_image_scaled = cv2.resize(adjusted_image, (frame_from_video.shape[1], frame_from_video.shape[0]))

    # Convert back to BGR and save the adjusted and scaled image
    cv2.imwrite("path/to/adjusted_image.jpg", cv2.cvtColor(adjusted_image_scaled, cv2.COLOR_RGB2BGR))
    print("Color space adjusted and image saved.")

def extract_first_frame(video_path, image_path, alpha=1.0, beta=0):
    video = cv2.VideoCapture(video_path)

    success, frame = video.read()
    if success:

        # Save the adjusted frame as an image
        cv2.imwrite(image_path, frame)
        print(f"Adjusted first frame saved as {image_path}")
    else:
        print("Error: Could not read the first frame")

    video.release()


video_file = "background.mp4"
image_file = "background.png"
adjust_color_space(video_file, image_file)

Color space adjusted and image saved.
